# Prepare the environment

In [ ]:
import os
import glob
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
import time
import cv2
import scipy
from PIL import Image
import matplotlib.gridspec as gridspec
from keras.layers import Dense, Conv2D, Conv2DTranspose, Dropout, Reshape, Input, merge
from keras.layers.core import Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model, load_model
from keras.optimizers import SGD, Adam, RMSprop
from keras.layers.advanced_activations import LeakyReLU
import matplotlib.pyplot as plt
from keras.utils.vis_utils import plot_model
import keras.backend as K
K.set_image_dim_ordering('tf')
from sklearn.utils import shuffle
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.initializers import RandomNormal
K.set_image_dim_ordering('tf')
config = tf.ConfigProto()
config.gpu_options.allow_growth = True 
set_session(tf.Session(config=config))
from collections import deque

# Import data and visualize a sample set

In [ ]:
filenames = glob.glob('data/animeface-character-dataset/thumb/*/*.png')

print("Number of images: ", len(filenames))

In [ ]:
plt.figure(figsize=(10,8))
for i in range(5):
    img = plt.imread(filenames[i], 0)
    plt.subplot(4, 5, i+1)
    plt.imshow(img)
    plt.title(img.shape)
    plt.xticks([])
    plt.yticks([])
plt.tight_layout()
plt.show()

# Preprocessing the images

In [ ]:
def norm_img(img):
    '''A function to normalize images
    Input:
        img: Original image as numpy array
    Output:
        img: Normalized image as numpy array
    '''
    return (img / 127.5) - 1

def denorm_img(img):
    '''A function to return the image to original form
    Input:
        img: Normalized image as numpy array
    Output:
        img: Image in original form as numpy array
    '''
    img = (img + 1) * 127.5
    return img.astype(np.uint8)

def sample_from_dataset(batch_size, image_shape, data_dir=None):
    '''Create a batch of image samples by sampling random images from a data directory.
    Resizes the image using image_shape and normalize the images.
    Input:
        batch_size : Sample size required
        image_size : Size that Image should be resized to
        data_dir : Path of directory where training images are placed.

    Output:
        sample : batch of processed images 
    '''
    sample_dim = (batch_size), + image_shape
    sample = np.empty(sample_dim, dtype=np.float32)
    all_data_dirlist = list(glob.glob(data_dir))
    sample_imgs_paths = np.random.choice(all_data_dirlist, batch_size)
    for index, img_filename in enumerate(sample_imgs_paths):
        image = Image.open(img_filename)
        image = image.resize(image_shape[:-1])
        image = image.convert('RGB')
        image = np.asarray(image)
        image = norm_img(image)
        sample[index,...] = image
    return sample

# Keras implemenation of DCGAN

## 1. Generate noise vector of Generator

In [ ]:
def gen_noise(batch_size, noise_shape):
    ''' Generates a numpy vector sampled from normal distribution of shape (batch_size,noise_shape)
    Input:
        batch_size : size of batch
        noise_shape: shape of noise vector, normally kept as 100 
    Output:a numpy vector sampled from normal distribution of shape (batch_size,noise_shape)     
    '''
    return np.random.normal(0, 1, size=(batch_size,)+noise_shape)

## 2. Create Generator architecture

In [ ]:
def get_gen_normal(noise_shape):
    '''This function takes an input the shape of the noise vector and creates the Keras generator architecture
    '''
    kernel_init = 'glorot_uniform'
    gen_input = Input(shape=noise_shape)
    
    # Transpose 2D Conv layer 1
    generator = Conv2DTranspose(filters=512, kernel_size=(4,4), strides=(1,1), padding="valid", data_format="channels_last", kernel_initializer=kernel_init)(gen_input)
    generator = BatchNormalization(momentum=0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    # Transpose 2D Conv layer 2
    generator = Conv2DTranspose(filters=256, kernel_size=(4,4), strides=(2,2), padding="same", data_format="channels_last", kernel_initializer=kernel_init)(generator)
    generator = BatchNormalization(momentum=0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    # Transpose 2D Conv layer 3
    generator = Conv2DTranspose(filters=128, kernel_size=(4,4), strides=(2,2), padding="same", data_format="channels_last", kernel_initializer=kernel_init)(generator)
    generator = BatchNormalization(momentum=0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    # Transpose 2D Conv layer 4
    generator = Conv2DTranspose(filters=64, kernel_size=(4,4), strides=(2,2), padding="same", data_format="channels_last", kernel_initializer=kernel_init)(generator)
    generator = BatchNormalization(momentum=0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    # 2D Conv layer 1
    generator = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding="same", data_format="channels_last", kernel_initializer=kernel_init)(generator)
    generator = BatchNormalization(momentum=0.5)(generator)
    generator = LeakyReLU(0.2)(generator)
    
    # Final Transpose 2D conv layer 5 to generate final image. Filter size 3 for 3 image channel
    generator = Conv2DTranspose(filters = 3, kernel_size = (4,4), strides = (2,2), padding = "same", data_format = "channels_last", kernel_initializer = kernel_init)(generator)
    
    # Tanh activation to get final normalized image
    generator = Activation("tanh")(generator)
    
    # Defining the optimizer and compiling the generator model
    gen_opt = Adam(lr=0.00015, beta_1=0.5)
    generator_model = Model(input=gen_input, output=generator)
    generator_model.compile(loss="binary_crossentropy", optimizer=gen_opt, metrics=["accuracy"])
    generator_model.summary()
    
    return generator_model